In [1]:
import json
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline

In [3]:
import gdown
file_id = '1uHtEkLYZwvZL1akgTfxWQkd7MeBCfdQM'
download_url = f"https://drive.google.com/uc?id={file_id}"
output = 'data.json'
gdown.download(download_url, output, quiet=False)
with open(output, 'r') as f:
    data = json.load(f)
print(data)

Downloading...
From: https://drive.google.com/uc?id=1uHtEkLYZwvZL1akgTfxWQkd7MeBCfdQM
To: /content/data.json
100%|██████████| 676k/676k [00:00<00:00, 60.8MB/s]

[{'Alert Title': 'Critical Patch Update - April 2024', 'Date': 'Rev 1, 16 April 2024', 'Link': 'https://www.oracle.com/security-alerts/cpuapr2024.html', 'Description': 'A Critical Patch Update is a collection of patches for multiple security vulnerabilities.   These patches address vulnerabilities in Oracle code and in third party components included in Oracle products.   These patches are usually cumulative, but each advisory describes only the security patches added since the previous Critical Patch Update Advisory. Thus, prior Critical Patch Update advisories should be reviewed for information regarding earlier published security patches.   Refer to  “Critical Patch Updates, Security Alerts and Bulletins”  for information about Oracle Security advisories.   These patches address vulnerabilities in Oracle code and in third party components included in Oracle products.   Refer to   for information about Oracle Security advisories.  Oracle continues to periodically receive reports of a

In [4]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = BertForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
for entry in data:
    alert_title = entry.get("Alert Title", "")
    description = entry.get("Description", "")
    full_text = alert_title + " " + description
    ner_results = ner_pipeline(full_text)
    #initialize
    classified_data = {
        "Product Name": None,
        "Product Version": None,
        "OEM Name": None,
        "Severity Level": None,
        "Vulnerability": None,
        "Mitigation Strategy": None,
        "Published Date": entry.get("Date", None),
        "Unique ID": None
    }
    for entity in ner_results:
        if "PRODUCT" in entity['entity_group']:
            classified_data["Product Name"] = entity['word']
        elif "ORG" in entity['entity_group']:
            classified_data["OEM Name"] = entity['word']
        elif "MISC" in entity['entity_group'] and "CVE" in entity['word']:
            classified_data["Unique ID"] = entity['word']
    classified_data["Severity Level"] = "High"  # Example value
    classified_data["Mitigation Strategy"] = "Check OEM website for patch."
    print("Processed Entry:")
    for key, value in classified_data.items():
        print(f"{key}: {value}")
    print("\n")

Processed Entry:
Product Name: None
Product Version: None
OEM Name: Oracle
Severity Level: High
Vulnerability: None
Mitigation Strategy: Check OEM website for patch.
Published Date: Rev 1, 16 April 2024
Unique ID: None


Processed Entry:
Product Name: None
Product Version: None
OEM Name: Matrix
Severity Level: High
Vulnerability: None
Mitigation Strategy: Check OEM website for patch.
Published Date: Rev 6, 28 May 2019
Unique ID: None


Processed Entry:
Product Name: None
Product Version: None
OEM Name: WebLogic Server
Severity Level: High
Vulnerability: None
Mitigation Strategy: Check OEM website for patch.
Published Date: Rev 6, 12 October 2020
Unique ID: None


Processed Entry:
Product Name: None
Product Version: None
OEM Name: Oracle
Severity Level: High
Vulnerability: None
Mitigation Strategy: Check OEM website for patch.
Published Date: Rev 3, 22 January 2020
Unique ID: None


Processed Entry:
Product Name: None
Product Version: None
OEM Name: Oracle
Severity Level: High
Vulnerabi